<a href="https://colab.research.google.com/github/Bearbrice/maDLC_MM_hands2_trained/blob/main/COLAB_maDLC2_VideoAnalysis2.1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# DeepLabCut 2.2 Toolbox - COLAB
![alt text](https://images.squarespace-cdn.com/content/v1/57f6d51c9f74566f55ecf271/1590444465547-SHXODUII311HEE407IL6/ke17ZwdGBToddI8pDm48kE4VnnB9_j2k1VP236ADqAFZw-zPPgdn4jUwVcJE1ZvWQUxwkmyExglNqGp0IvTJZUJFbgE-7XRK3dMEBRBhUpxQg9Vf0owGyf3dhfDKy8SxMujaKmp2B54Sb3VS1rO76Whq-cUhHVuKFlGUXsU9tJk/ezgif.com-video-to-gif.gif?format=1500w)

https://github.com/DeepLabCut/DeepLabCut

This notebook illustrates how to, for multi-animal projects, use the cloud-based GPU to:
- create a multi-animal training set
- train a network
- evaluate a network
- cross evaluate inference parameters
- analyze novel videos
- assemble tracklets
- create quality check plots

###This notebook assumes you already have a project folder with labeled data! 

This notebook demonstrates the necessary steps to use DeepLabCut for your own project.

This shows the most simple code to do so, but many of the functions have additional features, so please check out the docs on GitHub.

Mathis et al, in prep. <- please note, we are providing this toolbox as an early access release; more feeatures and details will be released with the forthcoming paper.




## First, go to "Runtime" ->"change runtime type"->select "Python3", and then select "GPU"


In [1]:
#(this will take a few minutes to install all the dependences!)
!pip install deeplabcut
%reload_ext numpy
%reload_ext scipy
%reload_ext matplotlib
%reload_ext mpl_toolkits

     |████████████████████████████████| 696kB 14.7MB/s 
     |████████████████████████████████| 29.2MB 177kB/s 
     |████████████████████████████████| 20.0MB 1.4MB/s 
     |████████████████████████████████| 296kB 42.4MB/s 
     |████████████████████████████████| 184kB 52.1MB/s 
     |████████████████████████████████| 3.1MB 35.5MB/s 
     |████████████████████████████████| 112kB 48.5MB/s 
     |████████████████████████████████| 21.6MB 57.4MB/s 
     |████████████████████████████████| 9.5MB 44.9MB/s 
     |████████████████████████████████| 13.1MB 179kB/s 
     |████████████████████████████████| 552kB 44.4MB/s 
  Created wheel for filterpy: filename=filterpy-1.4.5-cp37-none-any.whl size=110476 sha256=ee2459b94b2ebe1137e092902940ed444fe52a8858777a5d85066d3a9a237c26
  Stored in directory: /root/.cache/pip/wheels/c3/0c/dd/e92392c3f38a41371602d99fc77d6c1d42aadbf0c6afccdd02
  Created wheel for bayesian-optimization: filename=bayesian_optimization-1.2.0-cp37-none-any.whl size=11686 sha256=df6f

In [2]:
# Use TensorFlow 1.x:
%tensorflow_version 1.x

#GUIs don't work on the cloud, so we supress them:
import os
os.environ["DLClight"]="True"
import deeplabcut
deeplabcut.__version__

'2.1.10.4'

# GitHub

In [ ]:
# Clone the github repository
!git clone -l -s git://github.com/Bearbrice/maDLC_MM_hands2_trained.git
!ls

Cloning into 'maDLC_MM_hands2_trained'...
remote: Enumerating objects: 1534, done.
remote: Counting objects: 100% (5/5), done.
remote: Compressing objects: 100% (4/4), done.
remote: Total 1534 (delta 1), reused 5 (delta 1), pack-reused 1529
Receiving objects: 100% (1534/1534), 169.49 MiB | 30.77 MiB/s, done.
Resolving deltas: 100% (50/50), done.
maDLC_MM_hands2_trained  sample_data


# Drive google

In [ ]:
#Now, let's link to your GoogleDrive. Run this cell and follow the authorization instructions:
#(We recommend putting a copy of the github repo in your google drive if you are using the demo "examples")

from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


# Download

In [9]:
# !wget https://drive.google.com/file/d/1wrtFDOhth8J7HKuYZ_K1Joaqv2mnjetz/view?usp=sharing

!pip install gdown
!gdown https://drive.google.com/uc?id=16CD9S9LwTHifLPJ0dTAecmLRR7eDNpnB


Downloading...
From: https://drive.google.com/uc?id=16CD9S9LwTHifLPJ0dTAecmLRR7eDNpnB
To: /content/maDLC2_AfterTraining.zip
1.01GB [00:04, 213MB/s]


In [10]:
# Create a ZipFile Object and load sample.zip in it
from zipfile import ZipFile

with ZipFile('/content/maDLC2_AfterTraining.zip', 'r') as zipObj:
   # Extract all the contents of zip file in current directory
   zipObj.extractall()

## YOU WILL NEED TO EDIT THE PROJECT PATH **in the config.yaml file** TO BE SET TO YOUR GOOGLE DRIVE LINK!

Now, you can do this within COLAB! Simply navigate in the left panel to your project folder, double click on the config.yaml file, and you will see it load on the right! Edit the first part of your path, to match:

 /content/drive/My Drive/yourProjectFolderName


In [11]:
# PLEASE EDIT THIS:
VideoType = 'mp4' #, mp4, MOV, or avi, whatever you uploaded!

shuffle=1

# No need to edit this, we are going to assume you put videos you want to analyze in the "videos" folder, but if this is NOT true, edit below:
videofile_path = ['/content/maDLC_MM_hands2_trained/Inference_videos'] #Enter the list of videos or folder to analyze.
videofile_path

#No need to edit this, as you set it when you passed the ProjectFolderName (above): 
path_config_file = '/content/maDLC_MM_hands2_trained/maDLC_MM_hands-Brice-2021-07-08/config.yaml'
path_config_file
#This creates a path variable that links to your google drive project

'/content/maDLC_MM_hands2_trained/maDLC_MM_hands-Brice-2021-07-08/config.yaml'

## Start Analyzing videos: 
This function analyzes the new video. The user can choose the best model from the evaluation results and specify the correct snapshot index for the variable **snapshotindex** in the **config.yaml** file. Otherwise, by default the most recent snapshot is used to analyse the video.

The results are stored in hd5 file in the same directory where the video resides. 

In [12]:
print("Start Analyzing my video(s)!")
scorername = deeplabcut.analyze_videos(path_config_file, 
                                       videofile_path, 
                                       shuffle=shuffle, 
                                       videotype=VideoType, 
                                       c_engine=False)

Start Analyzing my video(s)!
Using snapshot-20000 for model /content/maDLC_MM_hands2_trained/maDLC_MM_hands-Brice-2021-07-08/dlc-models/iteration-0/maDLC_MM_handsJul8-trainset95shuffle1
Initializing ResNet
Activating extracting of PAFs
Analyzing all the videos in the directory...


  0%|          | 0/495 [00:00<?, ?it/s]

Starting to analyze %  /content/maDLC_MM_hands2_trained/Inference_videos/Pilot05-2_CUT.mp4
/content/maDLC_MM_hands2_trained/Inference_videos  already exists!
Loading  /content/maDLC_MM_hands2_trained/Inference_videos/Pilot05-2_CUT.mp4
Duration of video [s]:  16.5 , recorded with  30.0 fps!
Overall # of frames:  495  found with (before cropping) frame dimensions:  852 480
Starting to extract posture


500it [01:46,  4.70it/s]


Saving results in /content/maDLC_MM_hands2_trained/Inference_videos...


...

Saving results in /content/maDLC_MM_hands2_trained/Inference_videos...


  0%|          | 0/194 [00:00<?, ?it/s]

Starting to analyze %  /content/maDLC_MM_hands2_trained/Inference_videos/Video_Pers2_CUT.mp4
/content/maDLC_MM_hands2_trained/Inference_videos  already exists!
Loading  /content/maDLC_MM_hands2_trained/Inference_videos/Video_Pers2_CUT.mp4
Duration of video [s]:  6.47 , recorded with  30.0 fps!
Overall # of frames:  194  found with (before cropping) frame dimensions:  852 480
Starting to extract posture


200it [00:38,  5.17it/s]


Saving results in /content/maDLC_MM_hands2_trained/Inference_videos...
The videos are analyzed. Time to assemble animals and track 'em... 
 Call 'create_video_with_all_detections' to check multi-animal detection quality before tracking.
If the tracking is not satisfactory for some videos, consider expanding the training set. You can use the function 'extract_outlier_frames' to extract a few representative outlier frames.


## The steps below work on a single video at a time.
- Here you can create a video to check the pose estimation detection quality! If this looks good, proceed to tracklet conversions with the interactive GUI (ouside of COLAB for now), or if you know your optimal parameters, you can automate this and run the additional steps shown in a few cells down.

In [13]:
##### PROTIP: #####
## look at the output video; if the pose estimation (i.e. key points)
##  don't look good, don't proceed with tracking - add more data to your training set and re-train!

#let's check a specific video (PLEASE EDIT VIDEO PATH):
Specific_videofile = '/content/maDLC_MM_hands2_trained/Inference_videos/Pilot06-2_CUT.mp4'

# from deeplabcut.utils import auxiliaryfunctions
# scorername, DLCscorerlegacy = auxiliaryfunctions.GetScorerName(path_config_file, shuffle, trainFraction=0)
# print("scorename is: "+scorername)

deeplabcut.create_video_with_all_detections(path_config_file, [Specific_videofile], scorername)

#Again, if this does not look perfect, do not proceed! Retrain with more diverse data.

Creating labeled video for  Pilot06-2_CUT


100%|██████████| 718/718 [00:05<00:00, 133.47it/s]


## Convert Detections to Tracklets:

- The idea is that you test and adapt hyperparameters for tracking outside of COLAB. Once you have good parameters, this can be automated on future videos. Shown here!

- I.e., instead of always doing an interactive parameter setting step, you can simply convert tracklets to .h5 files using these parameters (see GitHub for more info).

In [14]:
#assemble tracklets:
#read the docs: which tracker to test out (you can run this many times to try multiple):
tracktype= 'box' #box, skeleton, ellipse

deeplabcut.convert_detections2tracklets(path_config_file, Specific_videofile, videotype=VideoType,
                                                    shuffle=shuffle, track_method=tracktype, overwrite=True)

Using snapshot-20000 for model /content/maDLC_MM_hands2_trained/maDLC_MM_hands-Brice-2021-07-08/dlc-models/iteration-0/maDLC_MM_handsJul8-trainset95shuffle1
Processing...  /content/maDLC_MM_hands2_trained/Inference_videos/Pilot06-2_CUT.mp4
/content/maDLC_MM_hands2_trained/Inference_videos  already exists!


0it [00:00, ?it/s]

Analyzing /content/maDLC_MM_hands2_trained/Inference_videos/Pilot06-2_CUTDLC_resnet50_maDLC_MM_handsJul8shuffle1_20000.h5


718it [00:14, 48.35it/s]

The tracklets were created. Now you can 'refine_tracklets'.


## Now you should manually verify the tracks and correct them if needed! [currently only working outside of COLAB]

In [ ]:
''' here is the code you would need:
os.environ["DLClight"]="False"
import deeplabcut

#ATTENTION:
picklefile = '/...._10000_bx.pickle' #(see your video folder for path i.e. right click and say copy path!!!)
vid ='/yourVIDEO.mp4'
#if you want occlusions filled in, tell us how many frames to fill in, i.e. if there is a gap in data:
framestofill = 0. #note, put "0" if you want ALL gaps filled!

%matplotlib inline

from deeplabcut import refine_tracklets
TrackletManager, TrackletVisualizer = refine_tracklets(path_config_file, 
                                                          picklefile, 
                                                          Specific_videofile, 
                                                          min_swap_frac=0,
                                                          min_tracklet_frac=0, 
                                                          max_gap=framestofill)
'''

## Let's assume you have great tracking parameters, and you want to analyze a full set of videos:

In [15]:
#^^^^^^^^^You do NOT neeed to run if you hit "save" in the GUI ^^^^^^^^^^
#this is just if you want to run the same parameters over a set of videos!

# You need to point to your pickle file, please "copy path" from the folder to the left (right click, copy path)
picklefile = '/content/maDLC_MM_hands2_trained/Inference_videos/Pilot06-2_CUTDLC_resnet50_maDLC_MM_handsJul8shuffle1_20000_bx.pickle' #(see your video folder for path i.e. right click and say copy path!!!)
vid ='/content/maDLC_MM_hands2_trained/Inference_videos/Pilot06-2_CUT.mp4'

deeplabcut.convert_raw_tracks_to_h5(path_config_file, picklefile)


100%|██████████| 2/2 [00:00<00:00, 239.80it/s]


In [18]:
deeplabcut.filterpredictions(path_config_file, 
                                 videofile_path, 
                                 videotype=VideoType, 
                                 track_method = tracktype)

Analyzing all the videos in the directory...
Filtering with median model /content/maDLC_MM_hands2_trained/Inference_videos/Pilot06-2_CUT.mp4
Data from Pilot06-2_CUT were already filtered. Skipping...
Filtering with median model /content/maDLC_MM_hands2_trained/Inference_videos/Pilot05-2_CUT.mp4
No unfiltered data file found in /content/maDLC_MM_hands2_trained/Inference_videos for video Pilot05-2_CUT and scorer DLC_resnet50_maDLC_MM_handsJul8shuffle1_20000 and box tracker.
Filtering with median model /content/maDLC_MM_hands2_trained/Inference_videos/Video_Pers2_CUT.mp4
No unfiltered data file found in /content/maDLC_MM_hands2_trained/Inference_videos for video Video_Pers2_CUT and scorer DLC_resnet50_maDLC_MM_handsJul8shuffle1_20000 and box tracker.
Filtering with median model /content/maDLC_MM_hands2_trained/Inference_videos/Pilot03-2.mp4
No unfiltered data file found in /content/maDLC_MM_hands2_trained/Inference_videos for video Pilot03-2 and scorer DLC_resnet50_maDLC_MM_handsJul8shuff

## Create plots of your trajectories:

In [19]:
deeplabcut.plot_trajectories(path_config_file,videofile_path, videotype=VideoType, track_method=tracktype)

Analyzing all the videos in the directory...
Loading  /content/maDLC_MM_hands2_trained/Inference_videos/Video_Pers2_CUT.mp4 and data.
No unfiltered data file found in /content/maDLC_MM_hands2_trained/Inference_videos for video Video_Pers2_CUT and scorer DLC_resnet50_maDLC_MM_handsJul8shuffle1_20000 and box tracker.
No detection data found in /content/maDLC_MM_hands2_trained/Inference_videos for video Video_Pers2_CUT, scorer DLC_resnet50_maDLC_MM_handsJul8shuffle1_20000, and tracker box
Make sure /content/maDLC_MM_hands2_trained/Inference_videos/Video_Pers2_CUT.mp4 was previously analyzed, and that detections were successively converted to tracklets using "deeplabcut.convert_detections2tracklets()" and "deeplabcut.convert_raw_tracks_to_h5()".
Loading  /content/maDLC_MM_hands2_trained/Inference_videos/Pilot06-2_CUT.mp4 and data.
/content/maDLC_MM_hands2_trained/Inference_videos/plot-poses/Pilot06-2_CUT  already exists!
Loading  /content/maDLC_MM_hands2_trained/Inference_videos/Pilot05-2_

Now you can look at the plot-poses file and check the "plot-likelihood.png" might want to change the "p-cutoff" in the config.yaml file so that you have only high confidnece points plotted in the video. i.e. ~0.8 or 0.9. The current default is 0.4. 

## Create labeled video:
This funtion is for visualiztion purpose and can be used to create a video in .mp4 format with labels predicted by the network. This video is saved in the same directory where the original video resides. 

In [20]:
deeplabcut.create_labeled_video(path_config_file,
                                videofile_path, 
                                shuffle=shuffle, 
                                draw_skeleton=True, 
                                videotype=VideoType, 
                                save_frames=False,
                                filtered=True, 
                                track_method = tracktype)

Analyzing all the videos in the directory...
/content/maDLC_MM_hands2_trained/Inference_videos  already exists!
Starting to process video: /content/maDLC_MM_hands2_trained/Inference_videos/Pilot06-2_CUT.mp4
/content/maDLC_MM_hands2_trained/Inference_videos  already exists!
Loading /content/maDLC_MM_hands2_trained/Inference_videos/Pilot06-2_CUT.mp4 and data.
Starting to process video: /content/maDLC_MM_hands2_trained/Inference_videos/Pilot05-2_CUT.mp4
Loading /content/maDLC_MM_hands2_trained/Inference_videos/Pilot05-2_CUT.mp4 and data.
No filtered data file found in /content/maDLC_MM_hands2_trained/Inference_videos for video Pilot05-2_CUT and scorer DLC_resnet50_maDLC_MM_handsJul8shuffle1_20000 and box tracker.
/content/maDLC_MM_hands2_trained/Inference_videos  already exists!
Starting to process video: /content/maDLC_MM_hands2_trained/Inference_videos/Video_Pers2_CUT.mp4
Loading /content/maDLC_MM_hands2_trained/Inference_videos/Video_Pers2_CUT.mp4 and data.
No filtered data file found 

100%|██████████| 718/718 [00:35<00:00, 20.25it/s]
